In [17]:
import gc
import pandas as pd
import geopandas as gpd
import sqlite3
from shapely import wkt
pd.set_option('display.max_columns', 100)

In [2]:
alg_conn = sqlite3.connect("/home/betorcha/Ml_projects/osm_data/osm_wilayats_sqlite/osm_Algiers.db")

In [12]:
df = pd.read_sql("select * from osm_Algiers", con= alg_conn)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428598 entries, 0 to 428597
Data columns (total 35 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ogc_fid       428598 non-null  int64  
 1   WKT_GEOMETRY  428598 non-null  object 
 2   osm_id        82066 non-null   object 
 3   name          23334 non-null   object 
 4   highway       48623 non-null   object 
 5   waterway      174 non-null     object 
 6   aerialway     10 non-null      object 
 7   barrier       2246 non-null    object 
 8   man_made      739 non-null     object 
 9   railway       649 non-null     object 
 10  z_order       49589 non-null   float64
 11  other_tags    73042 non-null   object 
 12  ref           221 non-null     object 
 13  address       0 non-null       object 
 14  is_in         0 non-null       object 
 15  place         424 non-null     object 
 16  type          1279 non-null    object 
 17  osm_way_id    346532 non-null  object 
 18  aero

In [14]:
df['geometry'] = gpd.GeoSeries.from_wkt(df['WKT_GEOMETRY'])
df = df.drop('WKT_GEOMETRY', axis = 1)
gdf = gpd.GeoDataFrame(df, geometry= 'geometry')
gdf.crs = 4326
del df
_ = gc.collect()

In [18]:
gdf.head()

,ogc_fid,osm_id,name,highway,waterway,aerialway,barrier,man_made,railway,z_order,other_tags,ref,address,is_in,place,type,osm_way_id,aeroway,amenity,admin_level,boundary,building,craft,geological,historic,land_area,landuse,leisure,military,natural,office,shop,sport,tourism,geometry
0,1,4366978,None,None,None,None,None,None,None,0.0,"""aeroway""=>""runway"",""length""=>""3500"",""ref""=>""0...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.25148 36.71294, 3.25098 36.71263..."
1,2,4366979,None,None,None,None,None,None,None,0.0,"""aeroway""=>""runway"",""length""=>""3500"",""ref""=>""0...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.20996 36.69116, 3.20993 36.69116..."
2,3,8142741,None,None,None,None,None,None,None,0.0,"""aeroway""=>""taxiway"",""ref""=>""G""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.25098 36.71263, 3.25107 36.71284..."
3,4,8142742,None,None,None,None,None,None,None,0.0,"""aeroway""=>""taxiway"",""ref""=>""F1""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.24024 36.70869, 3.24008 36.70884..."
4,5,8142744,None,None,None,None,None,None,None,0.0,"""aeroway""=>""taxiway"",""ref""=>""H1""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.21305 36.69290, 3.21289 36.69284..."


In [27]:
gdf

,ogc_fid,osm_id,name,highway,waterway,aerialway,barrier,man_made,railway,z_order,other_tags,ref,address,is_in,place,type,osm_way_id,aeroway,amenity,admin_level,boundary,building,craft,geological,historic,land_area,landuse,leisure,military,natural,office,shop,sport,tourism,geometry
0,1,4366978,None,None,None,None,None,None,None,0.0,"""aeroway""=>""runway"",""length""=>""3500"",""ref""=>""0...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.25148 36.71294, 3.25098 36.71263..."
1,2,4366979,None,None,None,None,None,None,None,0.0,"""aeroway""=>""runway"",""length""=>""3500"",""ref""=>""0...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.20996 36.69116, 3.20993 36.69116..."
2,3,8142741,None,None,None,None,None,None,None,0.0,"""aeroway""=>""taxiway"",""ref""=>""G""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.25098 36.71263, 3.25107 36.71284..."
3,4,8142742,None,None,None,None,None,None,None,0.0,"""aeroway""=>""taxiway"",""ref""=>""F1""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.24024 36.70869, 3.24008 36.70884..."
4,5,8142744,None,None,None,None,None,None,None,0.0,"""aeroway""=>""taxiway"",""ref""=>""H1""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (3.21305 36.69290, 3.21289 36.69284..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428593,5834,14664202,None,None,None,None,None,None,None,NaN,"""restriction""=>""no_left_turn""",None,None,None,None,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,GEOMETRYCOLLECTION (LINESTRING (3.24880 36.779...
428594,5835,14664203,None,None,None,None,None,None,None,NaN,"""restriction""=>""no_left_turn""",None,None,None,None,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,GEOMETRYCOLLECTION (LINESTRING (3.24880 36.779...
428595,5855,15599998,None,None,None,None,None,None,None,NaN,None,None,None,None,None,associatedStreet,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,GEOMETRYCOLLECTION (LINESTRING (3.25522 36.783...
428596,5860,15693593,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,GEOMETRYCOLLECTION (POINT (3.34015 36.74379))


In [28]:
gdf.other_tags

0         "aeroway"=>"runway","length"=>"3500","ref"=>"0...
1         "aeroway"=>"runway","length"=>"3500","ref"=>"0...
2                           "aeroway"=>"taxiway","ref"=>"G"
3                          "aeroway"=>"taxiway","ref"=>"F1"
4                          "aeroway"=>"taxiway","ref"=>"H1"
                                ...                        
428593                        "restriction"=>"no_left_turn"
428594                        "restriction"=>"no_left_turn"
428595                                                 None
428596                                                 None
428597                    "restriction"=>"only_straight_on"
Name: other_tags, Length: 428598, dtype: object